# Bin packing

In [1]:
%pip install opvious

## Formulation

In [2]:
import opvious.modeling as om

class BinPacking(om.Model):
    """Bin-packing MIP formulation"""
    
    items = om.Dimension()
    weight = om.Parameter.non_negative(items)
    bins = om.interval(1, om.size(items), name="B")
    max_weight = om.Parameter.non_negative()

    assigned = om.Variable.indicator(bins, items, qualifiers=['bins'])
    used = om.fragments.ActivationVariable(assigned, projection=1)

    @om.constraint
    def each_item_is_assigned_once(self):
        for i in self.items:
            yield om.total(self.assigned(b, i) for b in self.bins) == 1

    @om.constraint
    def bin_weights_are_below_max(self):
        for b in self.bins:
            bin_weight = om.total(self.weight(i) * self.assigned(b, i) for i in self.items)
            yield bin_weight <= self.max_weight()

    @om.objective
    def minimize_bins_used(self):
        return om.total(self.used(b) for b in self.bins)

model = BinPacking()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">BinPacking</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{items}&: I \\
  \S^p_\mathrm{weight}&: w \in \mathbb{R}_+^{I} \\
  \S^a&: B \doteq \{ 1 \ldots \# I \} \\
  \S^p_\mathrm{maxWeight}&: w^\mathrm{max} \in \mathbb{R}_+ \\
  \S^v_\mathrm{assigned[bins]}&: \alpha \in \{0, 1\}^{B \times I} \\
  \S^v_\mathrm{used}&: \psi \in \{0, 1\}^{B} \\
  \S^c_\mathrm{usedActivates}&: \forall b \in B, i \in I, \psi_{b} \geq \alpha_{b,i} \\
  \S^c_\mathrm{eachItemIsAssignedOnce}&: \forall i \in I, \sum_{b \in B} \alpha_{b,i} = 1 \\
  \S^c_\mathrm{binWeightsAreBelowMax}&: \forall b \in B, \sum_{i \in I} w_{i} \alpha_{b,i} \leq w^\mathrm{max} \\
  \S^o_\mathrm{minimizeBinsUsed}&: \min \sum_{b \in B} \psi_{b} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

In [3]:
import opvious

async def optimal_assignment(bin_max_weight, item_weights):
    """Returns a grouping of items which minimizes the number of bins used
    
    Args:
        bin_max_weight: The maximum allowable total weight for all items assigned to a given bin
        item_weights: Mapping from item name to its (non-negative) weight
    """
    problem = opvious.Problem(
        specification=model.specification(),
        parameters={'weight': item_weights, 'maxWeight': bin_max_weight},
    )
    solution = await opvious.Client.default().solve(problem)
    assignment = solution.outputs.variable('assigned')
    return list(assignment.reset_index().groupby('bins')['items'].agg(tuple))

## Example

In [4]:
await optimal_assignment(15, {
    'light': 5,
    'medium': 10,
    'heavy': 15,
})

[('light', 'medium'), ('heavy',)]